In [6]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import MetaTrader5 as mt5
from src.data_fetcher import initialize_mt5, get_data_from_mt5, shutdown_mt5
from src.strategy import add_moving_average_signals

if initialize_mt5():
    
    SYMBOL = "EURUSD"
    TIMEFRAME = mt5.TIMEFRAME_H1 # Timeframe de 1 heure
    N_BARS = 10000                # Récupérer 1000 barres

    df_raw = get_data_from_mt5(SYMBOL, TIMEFRAME, N_BARS)

    shutdown_mt5() 


    print("\n--- Données Brutes (dernières 5 lignes) ---")
    print(df_raw.tail())


    df_strategy = add_moving_average_signals(df_raw, short_window=50, long_window=200)
        
    print("\n--- Données avec Stratégie (dernières 10 lignes) ---")
    print(df_strategy.tail(10))

    signals = df_strategy[df_strategy['signal'] != 0]
    print("\n--- Derniers signaux générés ---")
    print(signals.tail())

else:
    print("Vérifiez que votre terminal MetaTrader 5 est bien lancé.")



Connexion réussie au compte #42820925 sur AdmiralsGroup-Demo
Données pour EURUSD récupérées avec succès (10000 barres).
Connexion à MetaTrader 5 fermée.

--- Données Brutes (dernières 5 lignes) ---
                        open     high      low    close  tick_volume  spread  \
time                                                                           
2025-11-07 12:00:00  1.15378  1.15516  1.15372  1.15498         1556       6   
2025-11-07 13:00:00  1.15497  1.15607  1.15491  1.15578         1981       6   
2025-11-07 14:00:00  1.15580  1.15649  1.15536  1.15557         2183       6   
2025-11-07 15:00:00  1.15558  1.15694  1.15554  1.15673         3195       6   
2025-11-07 16:00:00  1.15674  1.15785  1.15652  1.15749         3411       6   

                     real_volume  
time                              
2025-11-07 12:00:00            0  
2025-11-07 13:00:00            0  
2025-11-07 14:00:00            0  
2025-11-07 15:00:00            0  
2025-11-07 16:00:00            

In [7]:
import pandas as pd
import numpy as np

def run_simple_backtest(df_strategy, initial_cash=10000):
    print("Lancement du backtest simple (sans vectorbt)...")

    position = df_strategy['signal'].replace(0, np.nan).ffill().fillna(0)

    position[position == -1] = 0

    market_return = df_strategy['close'].pct_change()

    strategy_return = market_return * position.shift(1)


    cumulative_market = (1 + market_return).cumprod()
    cumulative_strategy = (1 + strategy_return.fillna(0)).cumprod()
    

    total_trades = (position.diff() == 1).sum()
    final_value = cumulative_strategy.iloc[-1] * initial_cash
    total_return_pct = (final_value - initial_cash) / initial_cash * 100
    
    print("\n--- Statistiques du Backtest Simple ---")
    print(f"Capital Initial: {initial_cash:,.2f}")
    print(f"Valeur Finale: {final_value:,.2f}")
    print(f"Retour Total: {total_return_pct:.2f}%")
    print(f"Nombre de trades (long only): {total_trades}")

    results = pd.DataFrame({
        'Buy_and_Hold': cumulative_market,
        'Strategy': cumulative_strategy
    })
    
    return results

# Exécuter le backtest simple

run_simple_backtest(df_strategy)

print("Module de stratégie chargé.")

Lancement du backtest simple (sans vectorbt)...

--- Statistiques du Backtest Simple ---
Capital Initial: 10,000.00
Valeur Finale: 10,539.88
Retour Total: 5.40%
Nombre de trades (long only): 31
Module de stratégie chargé.
